In [1]:
import os
import pandas as pd
import config_data
import datetime
from unidecode import unidecode
from urllib.parse import unquote

TARGET_LISTINGS_DIR = config_data.AIRBNB_DATA_DIR

In [2]:
def proc_dirto_row(loc_dir):
    #read all CSVs in loc_dir and retrieve the number of unique listings in each CSV
    timewise_dict = {}
    min_year =2020
    min_month = 12
    max_year = 0
    max_month = 1
    for csvfile in os.listdir(loc_dir):
        csvfile_fullpath = os.path.join(loc_dir,csvfile)
        if csvfile.split('.')[-1] == "csv":
            try:
                df = pd.read_csv(csvfile_fullpath)
                csvfile_no_ext = csvfile[:csvfile.rfind('.')]
                year, month, day = tuple(csvfile_no_ext.split('-')[-3:])
                year = int(year)
                month = int(month)
                if year <= min_year:
                    min_year = year
                    if month < min_month:
                        min_month = month
                if year >= max_year:
                    max_year = year
                    if month > max_month:
                        max_month = month
                k = (year, month, 1)
                timewise_dict[k]= len(pd.unique(df['id']))
#                 print(f"File Date:{(month,year)} Number of unique listings:{len(pd.unique(df['id']))}")
            except OSError:
                print(f"Skipping {csvfile_fullpath}")
    sorted_by_time = [(k,timewise_dict[k]) for k in sorted(timewise_dict.keys())]
    sorted_by_time = sorted_by_time[::-1]
    return sorted_by_time

In [3]:
def proc_dirto_df(loc_dir):
    #generate a df for the location with date, listings
    sorted_by_time = proc_dirto_row(loc_dir)
    proc_date_list = []
    proc_listing_list = []
    prev_month=0
    prev_year = 0
    year = 0
    month = 0
    prev_n_listings = 0
    curr_n_listings = 0
    for i in range(len(sorted_by_time)):
        prev_month = month
        prev_year = year
        prev_n_listings = curr_n_listings
        year=sorted_by_time[i][0][0]
        month=sorted_by_time[i][0][1]
        day = sorted_by_time[i][0][2]
        curr_n_listings = sorted_by_time[i][1]
        if not month == (prev_month-1) and prev_year == year:
            #interpolation of n_listings for missing months
            for j in range(month+1,prev_month):
                interp_month = j
                interp_date = str(year) + '-' + str(j) +'-'+str(day)
                prev_wt = (j - month) / (prev_month - month)
                curr_wt = 1 - prev_wt
                interp_listings = int(prev_wt * prev_n_listings + curr_wt * curr_n_listings)
                proc_date_list.append((interp_date, interp_listings))
                
        date = str(year) + '-' + str(month) +'-'+str(day)
        proc_date_list.append((date,curr_n_listings))
    df = pd.DataFrame(proc_date_list, columns = ['date','n_listings'])
    df = df.set_index('date')
    return df

In [6]:
cities = []
# for k, v in config_data.US_state_to_city.items():
#     for t in v:
#         cities.append(t)
# for k,v in config_data.country_to_city.items():
#     cities.append(v)
for v in config_data.misc_cities:
    cities.append(v)
print(cities)

['Beijing', 'Milan']


In [8]:
RAW_DATA_DIR = os.path.join('./AirbnbData/GzWithPrice')

for loc_dir in os.listdir(RAW_DATA_DIR):
    loc_dir_fullpath = os.path.join(RAW_DATA_DIR, loc_dir)
#     print(loc_dir_fullpath)
    mergedname = unidecode(unquote(loc_dir.replace("-", "")))
    for city in cities:
        citylower_nospace = city.lower().replace(" ","")
        if citylower_nospace in mergedname:
#             print(f"getting df for {city}")
            citydf = proc_dirto_df(loc_dir_fullpath)
            df = citydf
            #save the df
            citydir = os.path.join(TARGET_LISTINGS_DIR,city)
            if not os.path.exists(citydir):
                os.makedirs(citydir)
            df.to_csv(os.path.join(citydir,'listings.csv'))
            print(f"saved for {city}")

saved for Beijing
saved for Milan


In [ ]:
df = pd.read_csv

In [ ]:
import airbnb_dataset

myABData = airbnb_dataset.AirbnbDataset()

In [99]:
#for prices in beijing and milan

#find those id prices which are consistent throughout the data
cities_dirs = ["./AirbnbData/GzWithPrice/italy-lombardy-milan/","./AirbnbData/GzWithPrice/china-beijing-beijing/"]

#get listings that have been consistently present on Airbnb since start of 2019
#to be used to calculate average price
consistent_listings = set()

#get new listings that have been added per month
new_listings_all_months = dict()
consistent_listings = dict()

for citydir in cities_dirs:
    first_success = True
    currmonth_listings = []
    prevmonth_listings = []
    dates = []
    new_listings_this_month = 0
    prev_new_listings = 0
    month = 0
    year = 0
    prev_month = 0
    prev_year = 0
    for fname in sorted(os.listdir(citydir)):
        if "2019" in fname or "2020" in fname:
            name = fname.split('.')[0]
            name_split = name.split('-')
            prev_month = month
            prev_year = year
            day = 1#name_split[-1]
            month = int(name_split[-2])
            year = int(name_split[-3])
            date_val = str(year) + '-' + str(month) + '-' + str(day)
            prevmonth_listings = currmonth_listings
            df = pd.read_csv(os.path.join(citydir,fname))
            df = df.set_index(['id'])
            currmonth_listings = set(df.index.unique())
            if first_success:
                first_success = False
                consistent_listings[citydir.split('/')[-2]] = currmonth_listings
            else:
                prev_new_listings = new_listings_this_month
                new_listings_this_month = len(currmonth_listings - prevmonth_listings)
                consistent_listings[citydir.split('/')[-2]] &= currmonth_listings
            try:
                if not month == (prev_month+1) and prev_year == year:
                    print(prev_month,prev_new_listings)
                    print(month, new_listings_this_month)
                    #interpolation of prices for missing months
                    for j in range(prev_month+1, month):
                        interp_month = j
                        interp_date = str(year) + '-' + str(j) +'-'+str(day)
                        prev_wt = (j - month) / (prev_month - month)
                        curr_wt = 1 - prev_wt
                        interp_new_listings = int(prev_wt * prev_new_listings + curr_wt * new_listings_this_month)
                        new_listings_all_months[citydir.split('/')[-2]].append((interp_date,interp_new_listings))
                new_listings_all_months[citydir.split('/')[-2]].append((date_val,new_listings_this_month))
            except:
                new_listings_all_months[citydir.split('/')[-2]] = [(date_val,new_listings_this_month)]
        #create a dataframe of date mapping to new listings in that month
    new_fname = os.path.join(citydir,"newlistings.csv")
    print(new_listings_all_months)
    df2 = pd.DataFrame(new_listings_all_months[citydir.split('/')[-2]], columns = ['date','n_new_listings'])
    df2 = df2.set_index(["date"])
    df2.to_csv(new_fname)
    print(df2)
#     df2 = pd.Data

6 613
8 1244
{'italy-lombardy-milan': [('2019-1-1', 0), ('2019-2-1', 872), ('2019-3-1', 1390), ('2019-4-1', 3136), ('2019-5-1', 1099), ('2019-6-1', 759), ('2019-7-1', 1046), ('2019-8-1', 902), ('2019-9-1', 958), ('2019-10-1', 708), ('2019-11-1', 815), ('2019-12-1', 832), ('2020-1-1', 835), ('2020-2-1', 1312), ('2020-3-1', 0), ('2020-4-1', 1016), ('2020-5-1', 524), ('2020-6-1', 613), ('2020-7-1', 928), ('2020-8-1', 1244), ('2020-9-1', 738), ('2020-10-1', 664)]}
           n_new_listings
date                     
2019-1-1                0
2019-2-1              872
2019-3-1             1390
2019-4-1             3136
2019-5-1             1099
2019-6-1              759
2019-7-1             1046
2019-8-1              902
2019-9-1              958
2019-10-1             708
2019-11-1             815
2019-12-1             832
2020-1-1              835
2020-2-1             1312
2020-3-1                0
2020-4-1             1016
2020-5-1              524
2020-6-1              613
2020-7-1       

In [45]:
a = set([1,2,3])
b = set([1,2])
a&=b
a

{1, 2}

In [97]:
import numpy as np
for citydir in cities_dirs:
    cityname = citydir
    dates = []
    avg_prices = []
    prev_year = 0
    year = 0
    prev_month = 0
    prev_avg_price = 0
    avg_price = 0
    month = 0
    for fname in sorted(os.listdir(citydir)):
        if "2019" in fname or "2020" in fname:
            name = fname.split('.')[0]
            name_split = name.split('-')
            prev_year = year
            prev_month = month
            day = 1#name_split[-1]
            month = int(name_split[-2])
            year = int(name_split[-3])
            date_val = str(year)+'-'+str(month)+'-'+str(day)
            month = int(month)
            year = int(year)
            dates.append(date_val)
            consistent = list(consistent_listings[citydir.split('/')[-2]])
            df = pd.read_csv(os.path.join(citydir,fname))
            df = df.set_index(["id"])
            price_arr = df.loc[consistent]['price'].values
            price_float_arr = []
            for k in range(price_arr.shape[0]):
                price_float_arr.append(float(price_arr[k][1:].replace(',',"")))
            prev_avg_price = avg_price
            avg_price = np.average(price_float_arr)
            
            if not month == (prev_month+1) and prev_year == year:
                #interpolation of prices for missing months
                for j in range(prev_month+1, month):
                    interp_month = j
                    interp_date = str(year) + '-' + str(j) +'-'+str(day)
                    prev_wt = (j - month) / (prev_month - month)
                    curr_wt = 1 - prev_wt
                    interp_price = prev_wt * prev_avg_price + curr_wt * avg_price
                    avg_prices.append((interp_date, interp_price))
            avg_prices.append((date_val,avg_price))
            
    new_fname = os.path.join(citydir,"prices.csv")
    print(avg_prices)
    df2 = pd.DataFrame(avg_prices, columns = ['date','avg_price'])
    df2 = df2.set_index(["date"])
    df2.to_csv(new_fname)
    print(df2)

6 111.22712898125141
8 109.18040433702282
[('2019-1-1', 106.97526541676079), ('2019-2-1', 107.02789699570816), ('2019-3-1', 107.51863564490625), ('2019-4-1', 107.21899706347413), ('2019-5-1', 107.11328213236955), ('2019-6-1', 106.85227016037949), ('2019-7-1', 106.88547549130337), ('2019-8-1', 106.86288683081094), ('2019-9-1', 109.12977185452903), ('2019-10-1', 109.96792410210074), ('2019-11-1', 109.05398689857691), ('2019-12-1', 109.2510729613734), ('2020-1-1', 109.92048791506663), ('2020-2-1', 110.74757171899707), ('2020-3-1', 111.11011971990061), ('2020-4-1', 111.20973571267224), ('2020-5-1', 111.20487915066637), ('2020-6-1', 111.22712898125141), ('2020-7-1', 110.20376665913712), ('2020-8-1', 109.18040433702282), ('2020-9-1', 111.69196408402982), ('2020-10-1', 110.50783600632482)]
            avg_price
date                 
2019-1-1   106.975265
2019-2-1   107.027897
2019-3-1   107.518636
2019-4-1   107.218997
2019-5-1   107.113282
2019-6-1   106.852270
2019-7-1   106.885475
2019-8-1

In [104]:
DIR = './AirbnbData/Milan/'

df_listings = pd.read_csv(os.path.join(DIR,'listings.csv'))
df_listings = df_listings.set_index(['date'])
df_listings = df_listings.sort_index()
print(df_listings)
df_listings.to_csv(os.path.join(DIR,'listings.csv'))

           n_listings
date                 
2019-1-1        17723
2019-10-1       19967
2019-11-1       19786
2019-12-1       19944
2019-2-1        17823
2019-3-1        18318
2019-4-1        20626
2019-5-1        20627
2019-6-1        20675
2019-7-1        20765
2019-8-1        20908
2019-9-1        20322
2020-1-1        20015
2020-10-1       18674
2020-2-1        20280
2020-3-1        19201
2020-4-1        19728
2020-5-1        19961
2020-6-1        19922
2020-7-1        19352
2020-8-1        18783
2020-9-1        18763
